In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

%matplotlib inline

In [9]:
#path = r'C:\Users\adai\Documents\PD Study Data\updated_demo_pd_01\updated_demo_pd\01'
path = r'C:\Users\adai\Downloads\pd_study_v2_0_1004\PD Study v2.0\1004'

files = os.listdir(path)

for entries in files:
    if entries == 'annotations.csv':
        print(os.path.join(path, entries))
        df = pd.read_csv(os.path.join(path, entries))
        
### Annotations Processing
# removes unnecessary data columns
del df['Timestamp (ms)']
del df['AnnotationId']
del df['AuthorId']

df['Start Timestamp (ms)'] = pd.to_datetime(df['Start Timestamp (ms)'], unit='ms', utc=True).dt.tz_localize('UTC').dt.tz_convert('US/Central')
df['Stop Timestamp (ms)'] = pd.to_datetime(df['Stop Timestamp (ms)'], unit='ms', utc=True).dt.tz_localize('UTC').dt.tz_convert('US/Central')

testDay = df[df.EventType == 'Testing Day'].dropna(how='any').iloc[0]['Value'].replace(" ", "")

df = df[(df.EventType != 'Testing Day')]
del df['Value']

### Activity Sorting
sorter = df.EventType.unique()
sorterIndex = dict(zip(sorter,range(len(sorter))))

print(sorter)

df['EventType_Rank'] = df['EventType'].map(sorterIndex)
df['Cycle'] = df.groupby('EventType')['Start Timestamp (ms)'].rank(ascending=True).astype(int)
df[df['EventType'].str.contains('MDS-UPDRS')] = df[df['EventType'].str.contains('MDS-UPDRS')].replace(to_replace={'Cycle': {2: 3}})
df[df['EventType'].str.contains('Heart')] = df[df['EventType'].str.contains('Heart')].replace(to_replace={'Cycle': {1: 'NaN', 2: 'NaN'}})
df.sort_values(['EventType', 'EventType_Rank', 'Start Timestamp (ms)'], axis=0)

name = 'NewAnnotations_' + str(testDay) + '.xlsx'
writer = pd.ExcelWriter(name, options={'remove_timezone': True})
df.to_excel(writer,'Sheet1')

C:\Users\adai\Downloads\pd_study_v2_0_1004\PD Study v2.0\1004\annotations.csv
['Heart Rate Variability' 'MDS-UPDRS #1: Finger Tapping'
 'MDS-UPDRS #2: Hand Movements' 'MDS-UPDRS #3: Pronation-Supination'
 'MDS-UPDRS #4: Toe Tapping' 'MDS-UPDRS #5: Leg Agility'
 'MDS-UPDRS #6: Arising from Chair' 'MDS-UPDRS #7: Gait'
 'MDS-UPDRS #8: Postural Stability' 'MDS-UPDRS #9: Postural Hand Tremor'
 'MDS-UPDRS #11: Rest Tremor' 'Motor #1: Standing' 'Motor #2: Walking'
 'Motor #3: Walking while Counting' 'Motor #4: Finger to Nose'
 'Motor #5: Alternating Hand Movements' 'Motor #6: Sit to Stand'
 'Motor #7: Drawing on Paper' 'Motor #8: Typing on a Computer'
 'Motor #9: Nuts and Bolts' 'Motor #10: Drinking Water'
 'Motor #11: Organizing Folder' 'Motor #12: Folding Towels'
 'Motor #13: Sitting' 'MDS-UPDRS #10: Kinetic Hand Tremor']


In [ ]:
def activityCheck(activities):
# Checks the downloaded data from mc10cloud.com for missing or duplicate
# activity timestamps and returns missing and duplicate activity labels
#
# Inputs: activities - list of unique activities in the entire dataframe per day
#         testDay - desired testDay to scan
#
# Outputs: missing - list of missing activities per test day and cycle
#          duplicates - list of duplicate activity labels per test day and cycle
Complete = {'Heart Rate Variability', 'MDS-UPDRS #1: Finger Tapping',
           'MDS-UPDRS #2: Hand Movements', 'MDS-UPDRS #3: Pronation-Supination',
           'MDS-UPDRS #4: Toe Tapping', 'MDS-UPDRS #5: Leg Agility',
           'MDS-UPDRS #6: Arising from Chair', 'MDS-UPDRS #7: Gait',
           'MDS-UPDRS #8: Postural Stability', 'MDS-UPDRS #9: Postural Hand Tremor',
           'MDS-UPDRS #10: Kinetic Hand Tremor', 'MDS-UPDRS #11: Rest Tremor',
           'Motor #1: Standing', 'Motor #2: Walking', 'Motor #3: Walking while Counting',
           'Motor #4: Finger to Nose', 'Motor #5: Alternating Hand Movements',
           'Motor #6: Sit to Stand', 'Motor #7: Drawing on Paper',
           'Motor #8: Typing on a Computer', 'Motor #9: Nuts and Bolts',
           'Motor #10: Drinking Water', 'Motor #11: Organizing Folder',
           'Motor #12: Folding Towels', 'Motor #13: Sitting', 'Testing Day'}



In [ ]:
def merge_timestamps(df, duplicates):
# Inputs:  df - annotation dataframe
#          duplicates - list of activities to merge
#
# Outputs: df - updated dataframe with merged timestampes per activities
    for times in activities:
            
        
    

In [ ]:
def missing_timestamps(df, missing):
# Inputs:  df - annotation dataframe
#          missing - list of missing activity labels
#
# Outputs: df - updated dataframe with placeholder activities to indicate
#               the activity was missing
    for times in activities:
            